In [39]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

In [40]:
data = pd.read_csv('big_table.csv', sep=',')

In [41]:
data[25:35]

,Unnamed: 0,drug_cod,drug_x,provider_code,provider_drug_cod,code,drug_y
25,25,9953,Ацетилсалициловая к-та 500мг №10 тб,0000000000000000000013561,5163,13561,"Ацетилсалициловая к-та таб 0,5 г №10"
26,26,10604,Сульфокамфокаин 100мг/мл 2мл №10 амп р-р д/ин,0000000000000000000010250,5163,10250,Сульфокамфокаин амп 2 мл №10
27,27,11805,Пирацетам 1200мг №20 тб пл/о,0000000000000000000017469,5163,17469,Пирацетам таб 1200 мг № 20 п/о
28,28,3941,Торвакард 20мг №30 тб пл/о,0000000000000000000017256,5163,17256,"Торвакард 0,02 №30 таб. п\о"
29,29,6214,Фенистил гель 0.1% 50г (тубы),0000000000000000000015263,5163,15263,"Фенистил 0,1% гель 50 г"
30,30,5111,Бетасерк 24мг №20 тб,0000000000000000000014629,5163,14629,Бетасерк таб 24 мг №20
31,31,5132,Дибазол - УБФ 20мг №10 тб,0000000000000000000008789,5163,8789,Дибазол таб 20 мг №10
32,32,6229,Клотримазол 100мг №6 тб ваг,0000000000000000000010395,5163,10395,Клотримазол таб ваг 100 мг №6
33,33,4145,"Мидокалм Рихтер 100мг+2,5мг/мл 1мл №5 амп р-р...",0000000000000000000014400,5163,14400,"Мидокалм-Рихтер амп 100 мг+2,5 мг/мл 1 мл №5"
34,34,6191,Гевискон форте (мята) 150мл №1 сусп фл,0000000000000000000018330,5163,18330,Гевискон форте сусп фл 150 мл мятный


In [42]:
Parma_Sentence = data.drug_x.tolist()
Other_Sentence = data.drug_y.tolist()

In [43]:
display(Parma_Sentence[25:35],Other_Sentence[25:35])

[' Ацетилсалициловая к-та 500мг №10 тб',
 ' Сульфокамфокаин 100мг/мл 2мл №10 амп р-р д/ин',
 ' Пирацетам 1200мг №20 тб пл/о',
 ' Торвакард 20мг №30 тб пл/о',
 ' Фенистил гель 0.1% 50г (тубы)',
 ' Бетасерк 24мг №20 тб',
 ' Дибазол - УБФ 20мг №10 тб',
 ' Клотримазол 100мг №6 тб ваг',
 ' Мидокалм Рихтер 100мг+2,5мг/мл 1мл №5 амп р-р для в/в и в/м введ',
 ' Гевискон форте (мята) 150мл №1 сусп фл']

['Ацетилсалициловая к-та таб 0,5 г №10',
 'Сульфокамфокаин амп 2 мл №10',
 'Пирацетам таб 1200 мг № 20 п/о',
 'Торвакард 0,02 №30 таб. п\\о',
 'Фенистил  0,1% гель 50 г',
 'Бетасерк таб 24 мг №20',
 'Дибазол таб 20 мг №10',
 'Клотримазол таб ваг 100 мг №6',
 'Мидокалм-Рихтер  амп 100 мг+2,5 мг/мл 1 мл №5',
 'Гевискон форте сусп фл 150 мл мятный']

In [44]:
PM_chars = set()
for drug_line in Parma_Sentence:
    for ch in drug_line:
        if (ch not in PM_chars):
            PM_chars.add(ch)
PM_chars = sorted(list(PM_chars))    
PM_chars = set(PM_chars)
display(len(PM_chars))

141

In [45]:
Other_chars = set()
display(type(Other_chars))
for drug_line in Other_Sentence:
    for ch in drug_line:
        if (ch not in Other_chars):
            Other_chars.add(ch)
Other_chars = sorted(list(Other_chars))
Other_chars = set(Other_chars)
display(len(Other_chars))

set

151

In [46]:
#display(PM_chars, Other_chars)

In [47]:
display('Сравнение словарей:')
display('Нет в словаре ПМ:')
display(Other_chars - PM_chars)
display('Нет во внешнем словаре:')
display(type(PM_chars))
display(type(Other_chars))
delta = PM_chars - Other_chars
display(delta)
#display('Дилемма - добавлять ли во внешние словари символы ПМ? Они же могут там рано или поздно появиться.')
#display('А, давайте-ка добавим!')
#for ch in delta:
#    Other_chars.add(ch)
#display('Проверкочка:')
#delta = PM_chars - Other_chars
#display("Количество симовлов, которых нет во внешнем словаре:" + str(len(delta)))

'Сравнение словарей:'

'Нет в словаре ПМ:'

{'#', '=', '@', 'J', '\\', 'j', 'z', '\xa0', 'Щ', 'Ъ', 'Ы', 'Ь', '–'}

'Нет во внешнем словаре:'

set

set

{'&', '<', '>'}

In [48]:
PM_index_to_char_dict = {}
PM_char_to_index_dict = {}

for k, v in enumerate(PM_chars):
    PM_index_to_char_dict[k] = v
    PM_char_to_index_dict[v] = k

In [49]:
Other_index_to_char_dict = {}
Other_char_to_index_dict = {}

for k, v in enumerate(Other_chars):
    Other_index_to_char_dict[k] = v
    Other_char_to_index_dict[v] = k

In [50]:
max_len_PM_sent = max([len(line) for line in Parma_Sentence])
max_len_Other_sent = max([len(line) for line in Other_Sentence])

In [51]:
display(max_len_PM_sent)
display(max_len_Other_sent)

101

107

In [52]:
nb_samples = 1000
tokenized_PM_sentences    = np.zeros(shape = (nb_samples,max_len_PM_sent,   len(PM_chars)),    dtype='float32')
tokenized_Other_sentences = np.zeros(shape = (nb_samples,max_len_Other_sent,len(Other_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_Other_sent, len(Other_chars)),dtype='float32')

In [53]:
for i in range(nb_samples):
    for k,ch in enumerate(Parma_Sentence[i]):
        tokenized_PM_sentences[i,k,PM_char_to_index_dict[ch]] = 1
        
    for k,ch in enumerate(Other_Sentence[i]):
        tokenized_Other_sentences[i,k,Other_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,Other_char_to_index_dict[ch]] = 1

In [54]:
encoder_input = Input(shape=(None,len(PM_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]

In [55]:
decoder_input = Input(shape=(None,len(Other_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(Other_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)

In [56]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[tokenized_PM_sentences,tokenized_Other_sentences], 
          y=target_data,
          batch_size=264,
          epochs=100,
          validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 18s 23ms/step - loss: 1.3852 - val_loss: 1.1647
Epoch 2/100
800/800 [==============================] - 16s 20ms/step - loss: 0.9743 - val_loss: 1.0438
Epoch 3/100
800/800 [==============================] - 16s 20ms/step - loss: 0.9209 - val_loss: 1.0298
Epoch 4/100
800/800 [==============================] - 16s 19ms/step - loss: 0.9100 - val_loss: 1.0262
Epoch 5/100
800/800 [==============================] - 16s 20ms/step - loss: 0.9028 - val_loss: 1.0235
Epoch 6/100
800/800 [==============================] - 16s 20ms/step - loss: 0.8970 - val_loss: 1.0158
Epoch 7/100
800/800 [==============================] - 16s 20ms/step - loss: 0.8895 - val_loss: 1.0186
Epoch 8/100
800/800 [==============================] - 17s 21ms/step - loss: 0.8952 - val_loss: 1.0167
Epoch 9/100
800/800 [==============================] - 16s 20ms/step - loss: 0.8847 - val_loss: 0.9862
Epoch 10/100
800/800 [=====

800/800 [==============================] - 16s 20ms/step - loss: 0.5421 - val_loss: 0.7260
Epoch 80/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5381 - val_loss: 0.7311
Epoch 81/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5441 - val_loss: 0.7246
Epoch 82/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5335 - val_loss: 0.7147
Epoch 83/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5269 - val_loss: 0.7304
Epoch 84/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5323 - val_loss: 0.7116
Epoch 85/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5319 - val_loss: 0.7178
Epoch 86/100
800/800 [==============================] - 16s 20ms/step - loss: 0.5217 - val_loss: 0.7314
Epoch 87/100
800/800 [==============================] - 17s 22ms/step - loss: 0.5308 - val_loss: 0.7217
Epoch 88/100
800/800 [==============================] - 16s 20ms/step - loss:

In [57]:
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [58]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(Other_chars)))
    target_seq[0, 0,Other_char_to_index_dict[' ']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_Other_char = Other_index_to_char_dict[max_val_index]
        translated_sent += sampled_Other_char
        
        if ( (sampled_Other_char == ' ') or (len(translated_sent) > max_len_Other_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(Other_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent

In [60]:
for seq_index in range(10):
    inp_seq = tokenized_PM_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', Parma_Sentence[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence:  Имудон №24 тб д/расс
Decoded sentence: инарин 
-
Input sentence:  Глаумакс 0.005% 2.5мл фл-кап кап глаз
Decoded sentence: инарин 
-
Input sentence:  Эссенциале форте Н 300мг №30 капс
Decoded sentence: инарин 
-
Input sentence:  Эссенциале форте Н 300мг №30 капс
Decoded sentence: инарин 
-
Input sentence:  Димедрол 50мг №10 тб
Decoded sentence: инарин 
-
Input sentence:  Ибупрофен 5% 50г гель наруж 
Decoded sentence: инарин 
-
Input sentence:  Гевискон (мята) 300мл №1 сусп фл
Decoded sentence: инарин 
-
Input sentence:  Нурофен 100мг/5мл 150мл сусп д/приема внутрь апельсин
Decoded sentence: инарин 
-
Input sentence:  Нурофен 100мг/5мл 150мл сусп д/приема внутрь клубнич 
Decoded sentence: инарин 
-
Input sentence:  Нормодипин 10мг №30 тб
Decoded sentence: инарин 
